In [5]:
import pandas as pd
import random
import time
import re
import openai
import os

# Set OpenAI API key
openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Load gift data
gift_df = pd.read_excel('/content/Cadeaux_Produits (version 1).xlsx')

# Define the gift-related keywords
gift_keywords = [
    'gift', 'present', 'surprise'
]

# Define FAQ-related patterns with regex
faq_patterns = [
    (r".*how.*recommend.*gift.*", "🎁 I recommend gifts by asking you questions about the recipient, their interests, the occasion, and your budget. Then I match this info with my awesome gift database! ✨"),
    (r".*save.*preference.*", "💾 I'm working on that! Right now, I can't save preferences, but that's a feature coming soon! Stay tuned! 😉"),
    (r".*how.*sure.*gift.*like.*", "🤔 I can't be 100% sure, but I match gifts to interests, age, and occasion to give you the best options! 🎯"),
    (r".*more.*idea.*gift.*", "✨ Of course! Just let me know and I'll show you even more amazing gift ideas! 🎉"),
    (r".*refine.*result.*", "🔍 You can refine your search by giving me more details about the person, their hobbies, or adjusting your budget! 💡")
]

# Fun emojis and GIF links for more engaging interactions
fun_emojis = ['🎁', '✨', '🥳', '🎉', '🎈', '💝', '🛍️', '🎊']
gif_links = [
    "https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHJ1NXdxOXF0MnE0bGw4eXlzZDQwbW14OXd6eXg1NGV6eGUwNHRvOSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xTiTnh2PC1qEjDQUrC/giphy.gif",
    "https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExZjJtczl4bTJwdzczdmJvbGdkNzNmdDlsY2c5NHR2d2o5aXFzbmx0eSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/7nUkHEdz3AMkkbGWV1/giphy.gif",
]

last_results = pd.DataFrame()  # To store the last filtered results

def is_gift_related(message):
    return any(keyword in message.lower() for keyword in gift_keywords)

def is_faq_related(message):
    for pattern, response in faq_patterns:
        if re.search(pattern, message.lower()):
            return response
    return None

def show_gif():
    print(f"Check this out! {random.choice(gif_links)}")

def ask_gift_questions():
    print(f"{random.choice(fun_emojis)} Let's find the perfect gift for you! {random.choice(fun_emojis)}")
    recipient = input("👤 Who is the gift for? (adult, child, teen): ").lower()
    gender = input("🚻 What is the gender of the person? (male, female, unisex): ").lower()
    occasion = input("🎉 What is the occasion? (birthday, Christmas, charity, other): ").lower()
    relationship = input("🤝 What is your relationship with the person? (family, friends, colleagues, other): ").lower()
    interest = input("💡 What are their interests? (sports, music, reading, etc.): ").lower()
    budget = input("💸 What is your maximum budget? (in euros): ")
    return recipient, gender, occasion, relationship, interest, budget

def filter_gifts(recipient, gender, occasion, relationship, interest, budget):
    global last_results
    try:
        budget = float(budget)
    except ValueError:
        print("⚠️ Invalid budget. Using the maximum budget by default.")
        budget = gift_df['Prix (€)'].max()

    results = gift_df[
        (gift_df['Tranche d\'Âge'].str.lower() == recipient) &
        ((gift_df['Sexe'].str.lower() == gender) | (gift_df['Sexe'].str.lower() == 'unisexe')) &
        (gift_df['Event'].str.lower().str.contains(occasion, na=False)) &
        (gift_df['Scope'].str.lower().str.contains(relationship, na=False)) &
        (gift_df['Centres d\'Intérêt'].str.contains(interest, case=False, na=False)) &
        (gift_df['Prix (€)'] <= budget)
    ]
    last_results = results
    return results.head(5)

def recommend_gifts():
    recipient, gender, occasion, relationship, interest, budget = ask_gift_questions()
    results = filter_gifts(recipient, gender, occasion, relationship, interest, budget)
    if not results.empty:
        print(f"{random.choice(fun_emojis)} Here are some amazing gift ideas for you! {random.choice(fun_emojis)}")
        show_gif()
        time.sleep(1)
        for idx, row in results.iterrows():
            print(f"🎁 {row['Nom du Produit']}: {row['Description du Produit']} (💵 Price: {row['Prix (€)']}€)")
            time.sleep(0.5)
        ask_for_more_ideas()
    else:
        print("😔 Sorry, no gifts matched your criteria. Maybe you should try with different criteria!")
        show_gif()

def ask_for_more_ideas():
    more = input("Would you like to see more gift ideas? (yes/no): ").lower()
    if more == 'yes':
        show_more_gifts()

def show_more_gifts():
    global last_results
    more_results = last_results.iloc[5:]
    if not more_results.empty:
        print(f"{random.choice(fun_emojis)} Here are more gift ideas for you! {random.choice(fun_emojis)}")
        show_gif()
        for idx, row in more_results.iterrows():
            print(f"🎁 {row['Nom du Produit']}: {row['Description du Produit']} (💵 Price: {row['Prix (€)']}€)")
            time.sleep(0.5)
        print("✨ That's all the gift ideas matching your criteria! If nothing fits, you can refine your search. Would you like to refine your criteria? (yes/no)")
        refine = input().lower()
        if refine == 'yes':
            recommend_gifts()
        else:
            print("💡 What else can I help you with?")
            general_chat(input("You: "))
    else:
        print("✨ No more gift ideas available! ✨")

def general_chat(user_message):
    faq_response = is_faq_related(user_message)
    if faq_response:
        print(faq_response)
        show_gif()
    else:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": user_message}],
            max_tokens=150
        )
        print(response['choices'][0]['message']['content'].strip())

def main():
    print(f"{random.choice(fun_emojis)} Hello! I'm your personal assistant. Ask me anything! {random.choice(fun_emojis)}")
    while True:
        user_message = input("You: ")
        if 'exit' in user_message.lower():
            print(f"👋 Goodbye! Have a fantastic day! {random.choice(fun_emojis)}")
            break
        elif is_gift_related(user_message):
            recommend_gifts()
        else:
            general_chat(user_message)

if __name__ == "__main__":
    main()

🎁 Hello! I'm your personal assistant. Ask me anything! 🎈
You: Hello


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.